In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as opt

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


In [2]:
SEED=1

In [3]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [5]:
##########################
ARCH = models.resnet18() # set the architecture to RESNET 18
# please look up how to do that
########################
EPOCHS = 40
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=256
VAL_BATCH=128
WORKERS=4
# TRAINDIR="/workspace/data/imagenet2012/train"
# VALDIR="/workspace/data/imagenet2012/val"

In [6]:
TRAINDIR="/home/user1/work/w251/v3/week05/hw/imageNet-ILSVRC2012/download_and_prepare_imagenet_dataset/train"
VALDIR="/home/user1/work/w251/v3/week05/hw/imageNet-ILSVRC2012/download_and_prepare_imagenet_dataset/val"
TRAINDIR="/home/user1/data/train"
VALDIR="/home/user1/data/val"

### Check if cuda is available here

In [7]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!

torch.cuda.is_available()

True

### Assign your GPU below

In [8]:
# Assign your GPU in this cell
GPU = 0

In [9]:
# set your active device to your GPU in this cell
if torch.cuda.is_available():  
  dev =   "cuda:0"
else:  
  dev = "cpu"  

device = torch.device(dev)

In [10]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [11]:
def train(train_loader, model, criterion, optimizer, epoch, device=torch.device('cpu')):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        images = images.cuda(0, non_blocking=True)
        # send the target to cuda device
        target = target.cuda(0, non_blocking=True)
        
        #images = images.to(device)
        #target = target.to(device)
        
        # compute output
        output = model(images)

        # compute loss 
        loss = criterion(output, target)


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()
        
        ## backprop!
        ### loss... ???
        loss.backward()
        
        # update the weights!
        ## optimier .. ??
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [12]:
def validate(val_loader, model, criterion, device):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    # model ???
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            
            
            ### send the images and target to cuda
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)

            # compute loss
            loss  = criterion(output,target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [13]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [14]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [15]:
def save_checkpoint(state, is_best, filename='./checkpoint_192.pth.tar'):
    # save the model state!
    # state ??? 
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, './model_192_best.pth.tar')

In [16]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [17]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [18]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [19]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [20]:
IMG_SIZE = 32
IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [21]:
# select the model
model = ARCH

### Send the model to the cuda device

In [22]:
# send the model to the cuda device..
model = model.to(device)

### Instantiate the loss to cross entropy

In [23]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss()# .cuda(GPU)

### Instantiate the optimizer to SGD

In [24]:
# use SGD .. use the momentum and weight decay vars
optimizer = opt.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [25]:
# use CosineAnnealingLR
scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

### Create the train dataset object

In [26]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),# padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])


In [27]:
train_dataset = torchvision.datasets.ImageFolder(TRAINDIR, transform=transform_train)

### Create the val dataset object

In [28]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [29]:
val_dataset = torchvision.datasets.ImageFolder(VALDIR, transform=transform_val)

### Create the train dataloader

In [30]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=256, num_workers=WORKERS, shuffle=True)
#train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=128, shuffle=True)

### Create the c

In [31]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset,  batch_size=128, num_workers=WORKERS, shuffle=True) 
#val_loader = torch.utils.data.DataLoader(val_dataset,  batch_size=128, shuffle=True) 

In [32]:
best_acc1 = 0

In [ ]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, device)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, device)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][   0/5005]	Time 21.142 (21.142)	Data  8.075 ( 8.075)	Loss 7.0529e+00 (7.0529e+00)	Acc@1   0.39 (  0.39)	Acc@5   0.39 (  0.39)
Epoch: [0][  50/5005]	Time  1.659 ( 2.245)	Data  0.498 ( 0.852)	Loss 6.8925e+00 (7.0208e+00)	Acc@1   0.39 (  0.19)	Acc@5   1.56 (  0.95)
Epoch: [0][ 100/5005]	Time  1.474 ( 2.134)	Data  0.225 ( 0.851)	Loss 6.7341e+00 (6.9293e+00)	Acc@1   0.00 (  0.27)	Acc@5   3.52 (  1.27)
Epoch: [0][ 150/5005]	Time  3.739 ( 2.136)	Data  2.475 ( 0.873)	Loss 6.7555e+00 (6.8640e+00)	Acc@1   0.39 (  0.37)	Acc@5   2.34 (  1.60)
Epoch: [0][ 200/5005]	Time  1.317 ( 2.096)	Data  0.201 ( 0.850)	Loss 6.5548e+00 (6.8062e+00)	Acc@1   1.17 (  0.48)	Acc@5   3.12 (  1.93)
Epoch: [0][ 250/5005]	Time  1.383 ( 2.085)	Data  0.204 ( 0.850)	Loss 6.6010e+00 (6.7546e+00)	Acc@1   0.39 (  0.55)	Acc@5   2.73 (  2.20)
Epoch: [0][ 300/5005]	Time  1.424 ( 2.074)	Data  0.225 ( 0.847)	Loss 6.5084e+00 (6.7036e+00)	Acc@1   0.39 (  0.64)	Acc@5   4.69 (  2.45)
Epoch: [0][ 350/5005]	Time  2.942 ( 2.074

Epoch: [0][3000/5005]	Time  1.935 ( 2.053)	Data  0.772 ( 0.866)	Loss 4.5836e+00 (5.3701e+00)	Acc@1  14.45 (  7.13)	Acc@5  33.59 ( 18.71)
Epoch: [0][3050/5005]	Time  1.289 ( 2.054)	Data  0.199 ( 0.866)	Loss 4.6082e+00 (5.3556e+00)	Acc@1  13.67 (  7.25)	Acc@5  30.86 ( 18.95)
Epoch: [0][3100/5005]	Time  1.429 ( 2.053)	Data  0.253 ( 0.867)	Loss 4.3122e+00 (5.3402e+00)	Acc@1  18.75 (  7.38)	Acc@5  36.72 ( 19.20)
Epoch: [0][3150/5005]	Time  1.394 ( 2.054)	Data  0.274 ( 0.867)	Loss 4.2300e+00 (5.3261e+00)	Acc@1  18.75 (  7.49)	Acc@5  37.50 ( 19.43)
Epoch: [0][3200/5005]	Time  1.335 ( 2.055)	Data  0.218 ( 0.868)	Loss 4.3442e+00 (5.3118e+00)	Acc@1  19.14 (  7.62)	Acc@5  36.33 ( 19.67)
Epoch: [0][3250/5005]	Time  1.341 ( 2.054)	Data  0.188 ( 0.867)	Loss 4.3503e+00 (5.2977e+00)	Acc@1  14.45 (  7.74)	Acc@5  33.98 ( 19.91)
Epoch: [0][3300/5005]	Time  1.345 ( 2.053)	Data  0.224 ( 0.867)	Loss 4.3842e+00 (5.2837e+00)	Acc@1  15.62 (  7.86)	Acc@5  37.50 ( 20.14)
Epoch: [0][3350/5005]	Time  3.369 ( 2.054

Epoch: [1][ 600/5005]	Time  3.548 ( 2.027)	Data  2.334 ( 0.858)	Loss 3.8250e+00 (3.7838e+00)	Acc@1  22.27 ( 23.72)	Acc@5  49.22 ( 46.74)
Epoch: [1][ 650/5005]	Time  1.387 ( 2.024)	Data  0.239 ( 0.854)	Loss 3.7063e+00 (3.7798e+00)	Acc@1  25.78 ( 23.79)	Acc@5  47.27 ( 46.85)
Epoch: [1][ 700/5005]	Time  2.904 ( 2.023)	Data  1.710 ( 0.852)	Loss 3.9441e+00 (3.7770e+00)	Acc@1  23.05 ( 23.83)	Acc@5  43.75 ( 46.93)
Epoch: [1][ 750/5005]	Time  1.466 ( 2.022)	Data  0.218 ( 0.851)	Loss 3.5698e+00 (3.7748e+00)	Acc@1  25.78 ( 23.87)	Acc@5  53.52 ( 46.95)
Epoch: [1][ 800/5005]	Time  3.727 ( 2.025)	Data  2.631 ( 0.853)	Loss 3.6033e+00 (3.7680e+00)	Acc@1  28.52 ( 23.98)	Acc@5  50.00 ( 47.06)
Epoch: [1][ 850/5005]	Time  1.362 ( 2.023)	Data  0.188 ( 0.851)	Loss 3.6995e+00 (3.7642e+00)	Acc@1  22.27 ( 24.03)	Acc@5  45.31 ( 47.14)
Epoch: [1][ 900/5005]	Time  3.914 ( 2.021)	Data  2.763 ( 0.851)	Loss 3.5826e+00 (3.7580e+00)	Acc@1  29.69 ( 24.15)	Acc@5  52.34 ( 47.25)
Epoch: [1][ 950/5005]	Time  1.804 ( 2.021

Epoch: [1][3600/5005]	Time  1.370 ( 2.041)	Data  0.180 ( 0.863)	Loss 3.2636e+00 (3.5651e+00)	Acc@1  37.11 ( 27.08)	Acc@5  57.42 ( 51.04)
Epoch: [1][3650/5005]	Time  1.371 ( 2.041)	Data  0.205 ( 0.863)	Loss 3.4453e+00 (3.5627e+00)	Acc@1  32.03 ( 27.12)	Acc@5  50.00 ( 51.09)
Epoch: [1][3700/5005]	Time  1.335 ( 2.041)	Data  0.188 ( 0.863)	Loss 3.4276e+00 (3.5596e+00)	Acc@1  30.47 ( 27.16)	Acc@5  53.91 ( 51.15)
Epoch: [1][3750/5005]	Time  1.346 ( 2.041)	Data  0.188 ( 0.864)	Loss 3.0957e+00 (3.5566e+00)	Acc@1  31.25 ( 27.22)	Acc@5  57.42 ( 51.21)
Epoch: [1][3800/5005]	Time  1.394 ( 2.041)	Data  0.180 ( 0.863)	Loss 3.4661e+00 (3.5542e+00)	Acc@1  27.73 ( 27.25)	Acc@5  54.30 ( 51.26)
Epoch: [1][3850/5005]	Time  1.425 ( 2.041)	Data  0.196 ( 0.863)	Loss 3.3027e+00 (3.5511e+00)	Acc@1  32.42 ( 27.30)	Acc@5  55.08 ( 51.31)
Epoch: [1][3900/5005]	Time  1.320 ( 2.040)	Data  0.219 ( 0.863)	Loss 3.2471e+00 (3.5483e+00)	Acc@1  30.47 ( 27.34)	Acc@5  55.08 ( 51.36)
Epoch: [1][3950/5005]	Time  1.359 ( 2.040

Epoch: [2][1200/5005]	Time  4.188 ( 2.025)	Data  3.003 ( 0.849)	Loss 3.2818e+00 (3.1558e+00)	Acc@1  26.17 ( 33.51)	Acc@5  56.64 ( 58.54)
Epoch: [2][1250/5005]	Time  1.331 ( 2.026)	Data  0.236 ( 0.849)	Loss 3.0684e+00 (3.1541e+00)	Acc@1  36.72 ( 33.55)	Acc@5  60.16 ( 58.59)
Epoch: [2][1300/5005]	Time  2.508 ( 2.025)	Data  1.254 ( 0.849)	Loss 2.9180e+00 (3.1546e+00)	Acc@1  34.38 ( 33.55)	Acc@5  63.28 ( 58.57)
Epoch: [2][1350/5005]	Time  1.316 ( 2.024)	Data  0.221 ( 0.848)	Loss 3.3068e+00 (3.1538e+00)	Acc@1  32.03 ( 33.57)	Acc@5  53.12 ( 58.58)
Epoch: [2][1400/5005]	Time  2.510 ( 2.026)	Data  1.367 ( 0.850)	Loss 2.8948e+00 (3.1531e+00)	Acc@1  37.89 ( 33.58)	Acc@5  63.67 ( 58.61)
Epoch: [2][1450/5005]	Time  1.336 ( 2.024)	Data  0.193 ( 0.848)	Loss 3.4094e+00 (3.1515e+00)	Acc@1  30.08 ( 33.61)	Acc@5  53.12 ( 58.65)
Epoch: [2][1500/5005]	Time  1.631 ( 2.024)	Data  0.461 ( 0.848)	Loss 3.3572e+00 (3.1502e+00)	Acc@1  27.34 ( 33.64)	Acc@5  60.16 ( 58.69)
Epoch: [2][1550/5005]	Time  1.377 ( 2.026

...

Epoch: [5][   0/5005]	Time  8.014 ( 8.014)	Data  6.820 ( 6.820)	Loss 2.7034e+00 (2.7034e+00)	Acc@1  41.80 ( 41.80)	Acc@5  65.23 ( 65.23)
Epoch: [5][  50/5005]	Time  1.509 ( 2.136)	Data  0.197 ( 0.967)	Loss 2.6361e+00 (2.6722e+00)	Acc@1  44.14 ( 41.82)	Acc@5  67.97 ( 67.44)
Epoch: [5][ 100/5005]	Time  3.074 ( 2.104)	Data  1.888 ( 0.935)	Loss 2.5310e+00 (2.6806e+00)	Acc@1  41.80 ( 41.77)	Acc@5  70.31 ( 67.35)
Epoch: [5][ 150/5005]	Time  1.413 ( 2.064)	Data  0.170 ( 0.893)	Loss 2.3858e+00 (2.6907e+00)	Acc@1  48.83 ( 41.57)	Acc@5  70.31 ( 66.91)
Epoch: [5][ 200/5005]	Time  3.578 ( 2.069)	Data  2.458 ( 0.893)	Loss 2.5467e+00 (2.6954e+00)	Acc@1  44.92 ( 41.42)	Acc@5  69.92 ( 66.75)
Epoch: [5][ 250/5005]	Time  1.320 ( 2.054)	Data  0.169 ( 0.877)	Loss 2.6975e+00 (2.6988e+00)	Acc@1  42.19 ( 41.31)	Acc@5  67.58 ( 66.72)
Epoch: [5][ 300/5005]	Time  1.453 ( 2.046)	Data  0.229 ( 0.869)	Loss 2.3002e+00 (2.7005e+00)	Acc@1  46.48 ( 41.29)	Acc@5  72.66 ( 66.69)
Epoch: [5][ 350/5005]	Time  3.868 ( 2.047

Epoch: [5][3000/5005]	Time  1.547 ( 2.115)	Data  0.245 ( 0.891)	Loss 2.6920e+00 (2.7015e+00)	Acc@1  39.06 ( 41.37)	Acc@5  65.23 ( 66.55)
Epoch: [5][3050/5005]	Time  4.293 ( 2.118)	Data  3.050 ( 0.893)	Loss 2.6839e+00 (2.7010e+00)	Acc@1  47.27 ( 41.38)	Acc@5  68.75 ( 66.56)
Epoch: [5][3100/5005]	Time  1.381 ( 2.119)	Data  0.223 ( 0.893)	Loss 2.7882e+00 (2.7008e+00)	Acc@1  41.02 ( 41.38)	Acc@5  66.02 ( 66.56)
Epoch: [5][3150/5005]	Time  3.448 ( 2.120)	Data  2.242 ( 0.893)	Loss 2.7904e+00 (2.7001e+00)	Acc@1  38.67 ( 41.39)	Acc@5  64.06 ( 66.59)
Epoch: [5][3200/5005]	Time  1.393 ( 2.118)	Data  0.223 ( 0.892)	Loss 2.6783e+00 (2.7002e+00)	Acc@1  41.02 ( 41.39)	Acc@5  67.58 ( 66.58)
Epoch: [5][3250/5005]	Time  1.411 ( 2.117)	Data  0.207 ( 0.892)	Loss 2.6278e+00 (2.6997e+00)	Acc@1  43.75 ( 41.40)	Acc@5  67.19 ( 66.59)
Epoch: [5][3300/5005]	Time  1.304 ( 2.115)	Data  0.184 ( 0.891)	Loss 2.6953e+00 (2.6998e+00)	Acc@1  40.62 ( 41.40)	Acc@5  67.97 ( 66.59)
Epoch: [5][3350/5005]	Time  1.487 ( 2.115

Epoch: [6][ 600/5005]	Time  1.392 ( 2.019)	Data  0.268 ( 0.840)	Loss 2.6391e+00 (2.6311e+00)	Acc@1  39.45 ( 42.65)	Acc@5  67.58 ( 67.86)
Epoch: [6][ 650/5005]	Time  1.364 ( 2.019)	Data  0.165 ( 0.841)	Loss 2.6348e+00 (2.6354e+00)	Acc@1  41.02 ( 42.56)	Acc@5  68.75 ( 67.76)
Epoch: [6][ 700/5005]	Time  1.356 ( 2.018)	Data  0.198 ( 0.841)	Loss 2.4944e+00 (2.6367e+00)	Acc@1  44.92 ( 42.54)	Acc@5  70.31 ( 67.73)
Epoch: [6][ 750/5005]	Time  1.315 ( 2.022)	Data  0.177 ( 0.845)	Loss 2.7757e+00 (2.6380e+00)	Acc@1  39.84 ( 42.52)	Acc@5  64.84 ( 67.69)
Epoch: [6][ 800/5005]	Time  1.387 ( 2.018)	Data  0.212 ( 0.841)	Loss 2.5552e+00 (2.6394e+00)	Acc@1  43.75 ( 42.50)	Acc@5  68.75 ( 67.66)
Epoch: [6][ 850/5005]	Time  1.341 ( 2.017)	Data  0.220 ( 0.840)	Loss 2.7626e+00 (2.6406e+00)	Acc@1  36.33 ( 42.47)	Acc@5  66.41 ( 67.63)
Epoch: [6][ 900/5005]	Time  1.345 ( 2.015)	Data  0.200 ( 0.839)	Loss 2.5507e+00 (2.6420e+00)	Acc@1  47.27 ( 42.46)	Acc@5  68.36 ( 67.60)
Epoch: [6][ 950/5005]	Time  1.338 ( 2.012

Epoch: [6][3600/5005]	Time  1.400 ( 2.014)	Data  0.228 ( 0.839)	Loss 2.8307e+00 (2.6511e+00)	Acc@1  39.06 ( 42.34)	Acc@5  65.62 ( 67.42)
Epoch: [6][3650/5005]	Time  3.864 ( 2.014)	Data  2.755 ( 0.839)	Loss 2.7901e+00 (2.6507e+00)	Acc@1  41.41 ( 42.35)	Acc@5  64.45 ( 67.43)
Epoch: [6][3700/5005]	Time  1.388 ( 2.014)	Data  0.201 ( 0.839)	Loss 2.7740e+00 (2.6508e+00)	Acc@1  38.67 ( 42.34)	Acc@5  63.67 ( 67.42)
Epoch: [6][3750/5005]	Time  3.441 ( 2.014)	Data  2.286 ( 0.839)	Loss 2.4371e+00 (2.6510e+00)	Acc@1  44.92 ( 42.34)	Acc@5  69.53 ( 67.42)
Epoch: [6][3800/5005]	Time  1.378 ( 2.014)	Data  0.204 ( 0.839)	Loss 2.5502e+00 (2.6506e+00)	Acc@1  46.88 ( 42.34)	Acc@5  69.92 ( 67.43)
Epoch: [6][3850/5005]	Time  3.039 ( 2.014)	Data  1.944 ( 0.840)	Loss 2.5871e+00 (2.6501e+00)	Acc@1  40.62 ( 42.34)	Acc@5  67.58 ( 67.44)
Epoch: [6][3900/5005]	Time  2.562 ( 2.015)	Data  1.398 ( 0.840)	Loss 2.5876e+00 (2.6502e+00)	Acc@1  41.41 ( 42.33)	Acc@5  69.53 ( 67.44)
Epoch: [6][3950/5005]	Time  1.723 ( 2.014

Epoch: [7][1200/5005]	Time  2.199 ( 2.011)	Data  1.009 ( 0.841)	Loss 2.5239e+00 (2.5952e+00)	Acc@1  44.14 ( 43.19)	Acc@5  69.53 ( 68.47)
Epoch: [7][1250/5005]	Time  1.352 ( 2.012)	Data  0.182 ( 0.842)	Loss 2.4691e+00 (2.5953e+00)	Acc@1  45.31 ( 43.20)	Acc@5  72.27 ( 68.48)
Epoch: [7][1300/5005]	Time  1.526 ( 2.012)	Data  0.210 ( 0.842)	Loss 2.6913e+00 (2.5979e+00)	Acc@1  39.45 ( 43.14)	Acc@5  65.23 ( 68.41)
Epoch: [7][1350/5005]	Time  2.331 ( 2.011)	Data  1.182 ( 0.841)	Loss 2.6453e+00 (2.5982e+00)	Acc@1  42.97 ( 43.16)	Acc@5  69.14 ( 68.41)
Epoch: [7][1400/5005]	Time  1.428 ( 2.012)	Data  0.199 ( 0.843)	Loss 2.6989e+00 (2.5992e+00)	Acc@1  41.02 ( 43.15)	Acc@5  67.97 ( 68.39)
Epoch: [7][1450/5005]	Time  2.552 ( 2.012)	Data  1.375 ( 0.843)	Loss 2.5818e+00 (2.5997e+00)	Acc@1  41.41 ( 43.13)	Acc@5  69.53 ( 68.37)
Epoch: [7][1500/5005]	Time  1.315 ( 2.012)	Data  0.192 ( 0.842)	Loss 2.5613e+00 (2.6010e+00)	Acc@1  42.97 ( 43.09)	Acc@5  68.75 ( 68.36)
Epoch: [7][1550/5005]	Time  4.004 ( 2.013

In [ ]:
#from torchsummary import summary
print(model)

In [ ]:
print(device)